In [ ]:
# enable huge pages
# echo always >/sys/kernel/mm/transparent_hugepage/enabled

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_defaults()
sns.set_theme(style='whitegrid')

In [3]:
def bench(source='pointers', n=2**20, q=2**20, cc='clang++'):
    !{cc} -std=c++17 -O3 -march=native -D N={n} -D Q={q} {source}.cc -o run
    res = !./run
    update, query, check = map(float, res)
    check = int(check)
    print(f'{source:>18} {cc:<7} {n:<7} {q:<7} {check:10} {update} {query}')
    return update, query

In [36]:
bench('pointers')

          pointers clang++ 1048576 1048576    8237739 1126.16 1286.1


(1126.16, 1286.1)

In [57]:
bench('recursive')

         recursive clang++ 1048576 1048576    8237739 219.38 325.59


(219.38, 325.59)

In [38]:
bench('recursive2')

        recursive2 clang++ 1048576 1048576    8237739 215.75 197.86


(215.75, 197.86)

In [58]:
bench('while')

             while clang++ 1048576 1048576    8237739 193.98 139.37


(193.98, 139.37)

In [77]:
bench('iterative', q=2**22)

         iterative clang++ 1048576 4194304   32388439 85.28 170.9


(85.28, 170.9)

In [3]:
bench('iterative-branchless')
# todo: top-down branchless

iterative-branchless clang++ 1048576 1048576    8237739 84.88 183.72


(84.88, 183.72)

In [76]:
bench('iterative-unrolled', q=2**22)

iterative-unrolled clang++ 1048576 4194304   32388439 90.82 158.56


(90.82, 158.56)

In [9]:
bench('iterative')
bench('iterative-general')

         iterative clang++ 1048576 1048576    8237739 83.69 168.08
 iterative-general clang++ 1048576 1048576    8237739 84.03 171.86


(84.03, 171.86)

In [21]:
n = 2**20
bench('iterative-twoends', n=n)
bench('iterative', n=n)
bench('while', n=n)
bench('iterative-general', n=n)

 iterative-twoends clang++ 1048576 1048576    8237739 89.52 153.3
         iterative clang++ 1048576 1048576    8237739 87.5 148.76
             while clang++ 1048576 1048576    8237739 189.52 140.87
 iterative-general clang++ 1048576 1048576    8237739 84.88 145.63


(84.88, 145.63)

In [24]:
bench('iterative-branchless')

iterative-branchless clang++ 1048576 1048576    8237739 85.03 122.8


(85.03, 122.8)

In [7]:
bench('fenwick')
bench('fenwick2')
bench('fenwick-holes')

           fenwick clang++ 1048576 1048576    8237739 79.96 43.79
          fenwick2 clang++ 1048576 1048576    8237739 84.39 46.48
     fenwick-holes clang++ 1048576 1048576    8237739 70.56 45.78


(70.56, 45.78)

In [34]:
n = 10**7
q = 2**23
bench('fenwick', n=n, q=q)
bench('simd', n=n, q=q)

           fenwick clang++ 10000000 8388608   27577432 159.09 74.32
              simd clang++ 10000000 8388608   27577432 87.62 31.52


(87.62, 31.52)

In [33]:
bench('simd', n=n, q=q)

              simd clang++ 10000000 8388608   27577432 90.9 30.72


(90.9, 30.72)

In [70]:
n = 2**23
bench('iterative', n=n)
bench('iterative-branchless', n=n)

         iterative clang++ 8388608 1048576    7228362 144.5 202.48
iterative-branchless clang++ 8388608 1048576    7228362 138.98 224.31


(138.98, 224.31)

In [14]:
n = 2**14
bench('while', n=n)
bench('iterative-general', n=n)

             while clang++ 16384   1048576    1686941 45.77 88.35
 iterative-general clang++ 16384   1048576    1686941 15.52 115.19


(15.52, 115.19)

In [22]:
bench('refactor2', n=2**17)

         refactor2 clang++ 131072  1048576     829530 6.7463 2.0523


(6.7463, 2.0523)

In [23]:
bench('recursive', n=2**17)

         recursive clang++ 131072  1048576     829530 46.073 122.2992


(46.073, 122.2992)

In [18]:
bench('while', n=2**20)

             while clang++ 1048576 1048576    8237739 156.4913 123.1298


(156.4913, 123.1298)

In [15]:
bench('iterative', n=2**12)

         iterative clang++ 4096    1048576    8000795 4.6673 39.772


(4.6673, 39.772)

In [8]:
def plot(title=None, ylabel=None, path=None, ch=None, legend=True, ylim=None):
    if ch:
        lines = [
            (2**13, "32K"),
            (2**17, "512K"),
            (2**20, "4M"),
        ]

        for x, t in lines:
            plt.text(x * 1.2, ch, t)
            plt.axvline(x=x, color='black', linestyle='--')

    plt.xscale('log', basex=2)
    plt.xlabel('Array size')

    if ylabel:
        plt.ylabel(ylabel)

    if title:
        plt.title(title, pad=12)

    if legend:
        plt.legend(loc='best')

    plt.ylim(bottom=0, top=ylim)
    plt.margins(0)

    if path:
        fig = plt.gcf()
        fig.savefig(path)
    
    plt.show()

In [ ]:
ns = list(int(1.17**k) for k in range(30, 110))